- check this page to find the endpoint for getting data for "måndag"

- use dlt to create a database called stockholm_parking.duckdb, load data to a schema called staging and a table called parking_addresses

- in the table, create columns for the information below. Can you find out which keys should be used in the raw json data to access these information?
    - timestamp
    - address
    - city district
    - parking price

- use duckdblibrary in python to check if the data are loaded as expected. Follow the duckdb documentation in task 1

In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
import requests
import json
import dlt
import duckdb


API_KEY = dlt.secrets.get("api_secret_key")

# load_dotenv("sthlmAPI.env")

WEEKDAY="måndag"
# API_KEY = os.getenv("API_KEY")
BASE_URL = f"https://openparking.stockholm.se//LTF-Tolken/v1"
url=f"{BASE_URL}/ptillaten/weekday/{WEEKDAY}"

params = {
    "outputFormat": "json",
    "apikey": API_KEY
}

response = requests.get(url, params=params)
data = response.json()


def _parking_data():
    for feature in data["features"]:
        yield {
            "timestamp": data["timeStamp"],
            "address": feature["properties"]["ADDRESS"],
            "city district": feature["properties"]["CITY_DISTRICT"],
            "parking price": feature["properties"]["PARKING_RATE"]
        }


@dlt.resource(write_disposition="replace")
def append_data():
    yield from _parking_data()


def create_pipeline():
    pipeline = dlt.pipeline(
        pipeline_name="stockholm_parking_pipeline",
        destination=dlt.destinations.duckdb("stockholm_parking.duckdb"),
        dataset_name="staging"
    )
    
    pipeline.run(append_data, table_name="parking_addresses")
    return pipeline


pipeline = create_pipeline()

In [2]:
pipeline.dataset().parking_addresses.df()

,timestamp,address,city_district,parking_price,_dlt_load_id,_dlt_id
0,2025-04-03 13:11:23.244000+00:00,Schlytersvägen 51,Aspudden,"taxa 4: 10 kr/tim vardagar 7-19, dag före helg...",1743685884.3000658,Znp9aLLfcwzWnA
1,2025-04-03 13:11:23.244000+00:00,Schlytersvägen 39 - 38,Aspudden,"taxa 4: 10 kr/tim vardagar 7-19, dag före helg...",1743685884.3000658,rdxVs7jhVLIfkA
2,2025-04-03 13:11:23.244000+00:00,Floragatan 20 - 22,Östermalm,"taxa 3: 20 kr/tim vardagar 7-19, 15 kr/tim dag...",1743685884.3000658,Wh6nKDtfebZNzw
3,2025-04-03 13:11:23.244000+00:00,Oxenstiernsgatan 23 - Taptogatan 6,Östermalm,"taxa 3: 20 kr/tim vardagar 7-19, 15 kr/tim dag...",1743685884.3000658,IQqSkLUC5LRJMg
4,2025-04-03 13:11:23.244000+00:00,Oxenstiernsgatan 31 - 37,Östermalm,"taxa 3: 20 kr/tim vardagar 7-19, 15 kr/tim dag...",1743685884.3000658,HLY+sDGZR9ViFg
...,...,...,...,...,...,...
1755,2025-04-03 13:11:23.244000+00:00,Messeniusgatan 6 - 4,Fredhäll,"taxa 3: 20 kr/tim vardagar 7-19, 15 kr/tim dag...",1743685884.3000658,8c3RnmdtDPeHHw
1756,2025-04-03 13:11:23.244000+00:00,Messeniusgatan 6 - 4,Fredhäll,"taxa 3: 20 kr/tim vardagar 7-19, 15 kr/tim dag...",1743685884.3000658,DozDEfROU8z1hA
1757,2025-04-03 13:11:23.244000+00:00,Nyängsvägen 153,Stora Mossen,"taxa 5: 5 kr/tim vardagar 7-19, Boende: 300 kr...",1743685884.3000658,fUjvrbKYewHzPg
1758,2025-04-03 13:11:23.244000+00:00,Nyängsvägen 136,Stora Mossen,"taxa 5: 5 kr/tim vardagar 7-19, Boende: 300 kr...",1743685884.3000658,3T9PwsuKvialKQ


In [3]:
with duckdb.connect("stockholm_parking.duckdb") as con:
    result = con.execute("SELECT * FROM staging.parking_addresses").df()

result

,timestamp,address,city_district,parking_price,_dlt_load_id,_dlt_id
0,2025-04-03 15:11:23.244000+02:00,Schlytersvägen 51,Aspudden,"taxa 4: 10 kr/tim vardagar 7-19, dag före helg...",1743685884.3000658,Znp9aLLfcwzWnA
1,2025-04-03 15:11:23.244000+02:00,Schlytersvägen 39 - 38,Aspudden,"taxa 4: 10 kr/tim vardagar 7-19, dag före helg...",1743685884.3000658,rdxVs7jhVLIfkA
2,2025-04-03 15:11:23.244000+02:00,Floragatan 20 - 22,Östermalm,"taxa 3: 20 kr/tim vardagar 7-19, 15 kr/tim dag...",1743685884.3000658,Wh6nKDtfebZNzw
3,2025-04-03 15:11:23.244000+02:00,Oxenstiernsgatan 23 - Taptogatan 6,Östermalm,"taxa 3: 20 kr/tim vardagar 7-19, 15 kr/tim dag...",1743685884.3000658,IQqSkLUC5LRJMg
4,2025-04-03 15:11:23.244000+02:00,Oxenstiernsgatan 31 - 37,Östermalm,"taxa 3: 20 kr/tim vardagar 7-19, 15 kr/tim dag...",1743685884.3000658,HLY+sDGZR9ViFg
...,...,...,...,...,...,...
1755,2025-04-03 15:11:23.244000+02:00,Messeniusgatan 6 - 4,Fredhäll,"taxa 3: 20 kr/tim vardagar 7-19, 15 kr/tim dag...",1743685884.3000658,8c3RnmdtDPeHHw
1756,2025-04-03 15:11:23.244000+02:00,Messeniusgatan 6 - 4,Fredhäll,"taxa 3: 20 kr/tim vardagar 7-19, 15 kr/tim dag...",1743685884.3000658,DozDEfROU8z1hA
1757,2025-04-03 15:11:23.244000+02:00,Nyängsvägen 153,Stora Mossen,"taxa 5: 5 kr/tim vardagar 7-19, Boende: 300 kr...",1743685884.3000658,fUjvrbKYewHzPg
1758,2025-04-03 15:11:23.244000+02:00,Nyängsvägen 136,Stora Mossen,"taxa 5: 5 kr/tim vardagar 7-19, Boende: 300 kr...",1743685884.3000658,3T9PwsuKvialKQ


In [4]:
response.status_code

200